## 1.Counting the word in a Feed

In [1]:
import feedparser
import re

In [ ]:
#返回一个rss订阅源的标题 和包含的单词计数情况的字典
def getwordcounts(url):
    #解析订阅源
    d = feedparser.parse(url)
    wc = {}
    
    #循环遍历所有的文章条目
    for e in d.entries:
        if 'summary' in e: summary = e.summary
        else : summary = e.description
        
        #提取一个单词列表
        words = getwords(e.title + '' + summary)
        for word in words: #遍历所有单词
            wc.setdefault(word,0) #初始化词典的key和value
            wc[word] +=1  #单词的计数 +1
    return d.feed.title,wc  #返回文章标题，单词次数的字典

#提取字符
def  getwords (html):
    #去除所有HTML标记
    txt = re.compile(r'<[^>]+>').sub('',html)
    #利用所有非字母字符拆分出单词
    words = re.compile(r'^[A-Z^a-z]+').split(txt)
    #转化为小写
    return [word.lower() for word in words if word!='']

#feedlist.txt订阅源的URL

apcount = { } #所有url的词频字典
wordcounts = { } #每个url的词频字典 
feedlist = [ line  for line in file('/data/feedlist.txt')]
for feedurl in feedlist:
    title,wc = getwordcounts(feedurl)
    wordcounts[title] = wc #wordcounts{key:title,value:每个url的单词统计}
    for word,count in wc.items():#遍历每个URL的单词和其次数
        apcount.setdefault(word,0) #　生成总的统计字典
        if count > 1:
            apcount[word] += 1
            
#去除低频和高频词
wordlist =[]
for w,bc in apcount.items():
    frac = float(bc)/len(feedlist) 
    if frac>0.1 and frac < 0.5 : wordlist.append(w)
    

#输出记录每个单词统计情况的文本
out = file('/data/blogdata.txt','w')
out.write('Blog')
for word in wordlist:
    out.write('\t%s' % word)  #所有博客里单词的统计
out.write('\n')
for blogtitle,wc in wordcounts.items():
    out.write(blogtitle)
    for word in wordlist:
        if word in wc: out.write('\t%d' % wc[word])
        else : out.write('\t0')
    out.write('\n')
    